In [ ]:
from DocumentProcessing.pdf_processing import pdf_processor
from DocumentProcessing.pdf_processing import paper_processor

In [ ]:
pdf_paper = "/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/1810.04805.pdf"

In [ ]:
full_text, title,publish_display_data, authors, abstract, references = paper_processor.extract_text_from_pdf(pdf_paper)

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
full_text[1]

In [ ]:
pdf_processor.extract_toc_from_pdf(pdf_paper)

In [ ]:
full_text_2[3]

In [ ]:
pdf_processor.extract_pdf_metadata(pdf_paper)

In [ ]:
# pdf_processor.py

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        text = ''
        non_text_page_count = 0

        for page in doc:
            page_text = page.get_text()
            if not page_text:
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
            text += page_text

        return text

def extract_toc_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        toc = doc.get_toc(simple=False)
        non_text_page_count = 0

        # Check for non-text pages in the document
        for page in doc:
            if not page.get_text():
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
                break  # Break the loop after finding the first text page

        return toc


def extract_notes_from_pdf(pdf_path):
    notes = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            annotations = page.annots()
            if annotations:
                for annot in annotations:
                    annot_text = annot.info['content']
                    if annot_text:
                        notes.append(annot_text)
    return notes


def extract_pdf_metadata(pdf_path):
    with fitz.open(pdf_path) as doc:
        metadata = doc.metadata
    return metadata

# Example usage
if __name__ == "__main__":
    pass


def extract_pdf_metadata(file_path):
    """
    Extracts metadata from a PDF file.
    Args:
    file_path (str): Path to the PDF file.
    Returns:
    dict: Metadata of the PDF.
    """
    metadata = {}
    with pdfplumber.open(file_path) as pdf:
        metadata = pdf.metadata
    return metadata

In [ ]:
from DocumentManagement.Elastic import IndexSettingsGenerator,search_engine

In [ ]:
from elasticsearch import Elasticsearch, helpers

In [ ]:
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', 'dR8dVIqQ5=i3pPSH00zC'),  # Replace with your credentials
    verify_certs=True,
    ca_certs='/root/http_ca.crt'  # Path to your CA certificate
)


In [ ]:
from tika import parser

parsed = parser.from_file(pdf_paper)
print(parsed["metadata"])  # To print the metadata of the document.
print(parsed["content"])   # To print the text content of the document.


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)
    content = parsed['content']

    # Split the content by pages - Tika's plain text output may not provide clear page delimiters
    # Adjust the splitting logic based on the actual output format
    pages = content.split('some_page_delimiter')  # Replace 'some_page_delimiter' with the actual delimiter used by Tika

    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, start=1):
        page_text = page_content.strip()

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("10 consecutive pages without text found. PDF may contain only images. Please use OCR.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text


In [ ]:
pdf_text = extract_text_tika(pdf_paper)

In [ ]:
pdf_paper

In [ ]:
pdf_text

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

def add_invisible_markers(pdf_path, marker):
    reader = PdfReader(pdf_path)
    writer = PdfWriter()

    for i in range(len(reader.pages)):
        page = reader.pages[i]
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        can.setFontSize(1)  # Setting font size to 1 or 0 to make it nearly invisible
        can.drawString(0, 0, marker)  # Position at bottom-left; can adjust as needed
        can.save()

        packet.seek(0)
        new_pdf = PdfReader(packet)
        page.merge_page(new_pdf.pages[0])
        writer.add_page(page)

    with open("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "wb") as f:
        writer.write(f)

add_invisible_markers(pdf_paper ,"__PageBreak__12345__")



In [ ]:
from tika import parser

def extract_pages_with_tika(file_path, marker):
    parsed = parser.from_file(file_path)
    content = parsed['content']
    pages = content.split(marker)  # Splitting text using the marker

    text_by_page = {i+1: page.strip() for i, page in enumerate(pages)}
    return text_by_page

text_by_page = extract_pages_with_tika("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "__PageBreak__12345__")


In [ ]:
text_by_page

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
text_by_page[1], full_text[1]

In [ ]:
from DocumentIndexing.MongoDB import documentstore

In [ ]:
res = documentstore.get_document_from_collection('research_paper', True)

In [ ]:
res

In [ ]:
import pickle
PAGE = "PDF VIEWER"
pickle.dump( PAGE, open(CHAT_BOT_STATUT_CACHE_PATH, "wb" ) )

In [ ]:
with open (CHAT_BOT_STATUT_CACHE_PATH, 'rb') as fp:
    itemlist = pickle.load(fp)

In [ ]:
CHAT_BOT_STATUT_CACHE_PATH = "/root/gpt_projects/ABoringKnowledgeManagementSystem/WebApp/WebUI/static/chat_bot_statut_cache/chat_bot_statut_cache.pkl"


In [ ]:
PDF_VIEWER_CACHE_PATH = "/root/gpt_projects/ABoringKnowledgeManagementSystem/WebApp/WebUI/static/pdf_viewer_cache/pdf_viewer_cache.pkl"

In [ ]:
PDF_FILE = ('d42a0cd9-fdfd-4639-a38c-a85b08cd37a7','research_paper')

In [ ]:
pickle.dump( PDF_FILE, open(PDF_VIEWER_CACHE_PATH, "wb" ) )

In [1]:
from DocumentIndexing.Elastic import IndexSettingsGenerator
from DocumentIndexing.Elastic import search_engine
from elasticsearch.helpers import scan

In [2]:
from config import *

In [3]:
from DocumentIndexing.Embedding.embedding import TextEmbedder

/root/codebox39/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from DocumentIndexing.Elastic.search_engine import SearchEngine

In [ ]:
search_engine = SearchEngine()

In [ ]:
search_engine.search_for_terms('research_paper_chunk_level','bert')

In [ ]:
from Chat import rag

In [ ]:
res = rag.two_step_retrival_page("what is BERT?","2610a7e7-359f-4e79-85d8-d3e3dff51dab", "research_paper",return_top_n=3)

In [ ]:
res

In [ ]:
print(rag.two_step_prompt_page(res))

In [ ]:
prompt = rag.one_document_prompt(res)

In [ ]:
print(prompt)

In [4]:
indexsetting = IndexSettingsGenerator.GeneralIndexSettings(EMBEDDING_DINENSION)
chunk_mapping = indexsetting.chunk_level_mapping()
document_mapping = indexsetting.document_level_mapping()
page_mapping = indexsetting.page_level_mapping()

In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)

/tmp/ipykernel_191312/2026850524.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)


In [6]:
new_search_engine = search_engine.SearchEngine()    
for mapping,level in zip([chunk_mapping, page_mapping,document_mapping],DOCUMENT_LEVEL):
    print(mapping)
    response = es.indices.delete(index=f'research_paper_'+level, ignore=[400, 404])
    new_search_engine.create_index(index_name=f'research_paper_'+level, index_settings=mapping)
    


{'mappings': {'properties': {'document_id_elastic': {'type': 'keyword'}, 'document_id_universal': {'type': 'keyword'}, 'page_number': {'type': 'integer'}, 'chunk_number': {'type': 'integer'}, 'upload_date': {'type': 'date', 'format': 'epoch_millis'}, 'max_split_number': {'type': 'integer'}, 'text_piece': {'type': 'text'}, 'language': {'type': 'keyword'}, 'text_piece_vector': {'type': 'dense_vector', 'dims': 512}, 'metadata': {'type': 'object', 'dynamic': True}, 'document_tags': {'type': 'keyword'}, 'related_documents': {'type': 'object', 'dynamic': True}, 'acheived': {'type': 'boolean'}}}}


/tmp/ipykernel_191312/1163721322.py:4: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.delete(index=f'research_paper_'+level, ignore=[400, 404])


{'mappings': {'properties': {'document_id_elastic': {'type': 'keyword'}, 'document_id_universal': {'type': 'keyword'}, 'page_number': {'type': 'integer'}, 'upload_date': {'type': 'date', 'format': 'epoch_millis'}, 'max_split_number': {'type': 'integer'}, 'text_piece': {'type': 'text'}, 'language': {'type': 'keyword'}, 'page_summary': {'type': 'text'}, 'page_smmary_vector': {'type': 'dense_vector', 'dims': 512}, 'text_piece_vector': {'type': 'dense_vector', 'dims': 512}, 'metadata': {'type': 'object', 'dynamic': True}, 'document_tags': {'type': 'keyword'}, 'related_documents': {'type': 'object', 'dynamic': True}, 'acheived': {'type': 'boolean'}}}}
{'mappings': {'properties': {'document_id_elastic': {'type': 'keyword'}, 'document_id_universal': {'type': 'keyword'}, 'upload_date': {'type': 'date', 'format': 'epoch_millis'}, 'max_page_number': {'type': 'integer'}, 'document_title': {'type': 'keyword'}, 'document_summary': {'type': 'text'}, 'language': {'type': 'keyword'}, 'document_title_v

In [ ]:
eleasticengine = search_engine.SearchEngine()

In [ ]:
for index_name in DOCUMENT_TYPE:
    eleasticengine.create_index(index_name,mapping)

In [ ]:
index_name = "research_paper_chunk_level"

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Connect to Elasticsearch
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)
# Index name
index_name = "research_paper"

# Initialize the scan
results = scan(es, index=index_name, query={"query": {"match_all": {}}})

# Iterate over the results
for doc in results:
    print(doc)


In [ ]:
from config import DOCUMENT_TYPE, ES_HOST, ES_HTTP_AUTH, ES_CA_CERTS_PATH, EMBEDDING_DINENSION,DOCUMENT_LEVEL

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)


existing_indices = DOCUMENT_TYPE
# Delete each index
for index in existing_indices:
    response = es.indices.delete(index=index)
    print(f"Deleted index: {index}")

In [ ]:
res['hits']['hits']

In [ ]:
vector = embedding_local.embeddings_multilingual("ﬁnetuning")

In [ ]:
res2 = eleasticengine.vector_search("research_paper",vector)

In [ ]:
res2['hits']

In [ ]:
import pdftitle
print(pdftitle.__file__)


In [ ]:
from pdftitle import get_title_from_file

In [ ]:
pdf_path = '/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/416f1ec4-bcbd-4204-9069-b6ea86c1b7a9.pdf'

In [ ]:
try:
    title = get_title_from_file(fp)
    print(f"The extracted title is: {title}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
file_path = 'DocumentBank/research_paper/2610a7e7-359f-4e79-85d8-d3e3dff51dab.pdf'

In [ ]:
fp= "/root/downloads/MY_CV_Final.pdf"

In [ ]:
from DocumentManagement.documents import Document

In [ ]:
doc = Document(file_path = 'DocumentBank/research_paper/2610a7e7-359f-4e79-85d8-d3e3dff51dab.pdf',document_type='research_paper',document_id='123')

In [ ]:
from DocumentProcessing.pdf_processing import pdf_processor

In [ ]:
pdf_processor.extract_pdf_metadata(file_path)

In [ ]:
pdf_processor.structured_metadata_for_paper(file_path)

In [ ]:
from llama_index import SimpleDirectoryReader


In [ ]:
from pathlib import Path
from llama_hub.file.unstructured import UnstructuredReader

loader = UnstructuredReader()
documents = loader.load_data(file=file_path)

In [ ]:
from llama_index import SimpleDirectoryReader
from llama_index.ingestion import IngestionPipeline
from llama_index.node_parser import TokenTextSplitter

In [ ]:
pipeline = IngestionPipeline(transformations=[TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
])

In [ ]:
from llama_index.text_splitter import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
nodes = pipeline.run(documents=documents)

In [ ]:
from llama_index.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)


In [ ]:
documents

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex(nodes)


In [ ]:
from config import *    

In [ ]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)

In [ ]:
from llama_index.embeddings.elasticsearch import ElasticsearchEmbedding
from llama_index.vector_stores import ElasticsearchStore
from llama_index import ServiceContext, StorageContext, VectorStoreIndex



from llama_index.vector_stores import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name= index_name,
    es_client=es,     
    vwctor_field="text_piece_vector",  
)

In [ ]:
text = 'We also undertake systematic study of “data contamination” – growing problem when training high capacity models\non datasets such as Common Crawl which can potentially include content from test datasets simply because such\ncontent often exists on the web In this paper we develop systematic tools to measure data contamination and quantify\nits distorting effects Although we ﬁnd that data contamination has minimal effect on GPT’ performance on most\ndatasets we do identify few datasets where it could be inﬂating results and we either do not report results on these\ndatasets or we note them with an asterisk depending on the severity\nIn addition to all the above we also train series of smaller models ranging from 125 million parameters to 13 billion\nparameters in order to compare their performance to GPT in the zero one and fewshot settings Broadly for most\ntasks we ﬁnd relatively smooth scaling with model capacity in all three settings one notable pattern is that the gap\nbetween zero one and fewshot performance often grows with model capacity perhaps suggesting that larger models\nare more proﬁcient metalearners\nFinally given the broad spectrum of capabilities displayed by GPT we discuss concerns about bias fairness and\nbroader societal impacts and attempt preliminary analysis of GPT’ characteristics in this regard\nThe remainder of this paper is organized as follows In Section we describe our approach and methods for training\nGPT and evaluating it Section presents results on the full range of tasks in the zero one and fewshot settings\nSection addresses questions of data contamination traintest overlap Section discusses limitations of GPT\nSection discusses broader impacts Section reviews related work and Section concludes\n Approach\nOur basic pretraining approach including model data and training is similar to the process described in RWC19\nwith relatively straightforward scaling up of the model size dataset size and diversity and length of training Our use\nof incontext learning is also similar to RWC19 but in this work we systematically explore different settings for\nlearning within the context Therefore we start this section by explicitly deﬁning and contrasting the different settings\nthat we will be evaluating GPT on or could in principle evaluate GPT on These settings can be seen as lying on \nspectrum of how much taskspeciﬁc data they tend to rely on Speciﬁcally we can identify at least four points on this\nspectrum see Figure for an illustration\n• FineTuning FT has been the most common approach in recent years and involves updating the weights of\n pretrained model by training on supervised dataset speciﬁc to the desired task Typically thousands to\nhundreds of thousands of labeled examples are used The main advantage of ﬁnetuning is strong performance\non many benchmarks The main disadvantages are the need for new large dataset for every task the potential\nfor poor generalization outofdistribution MPL19 and the potential to exploit spurious features of the\ntraining data GSL18 NK19 potentially resulting in an unfair comparison with human performance In\nthis work we do not ﬁnetune GPT because our focus is on taskagnostic performance but GPT can be\nﬁnetuned in principle and this is promising direction for future work\n• FewShot FS is the term we will use in this work to refer to the setting where the model is given few\ndemonstrations of the task at inference time as conditioning RWC19 but no weight updates are allowed\nAs shown in Figure for typical dataset an example has context and desired completion for example\nan English sentence and the French translation and fewshot works by giving examples of context and\ncompletion and then one ﬁnal example of context with the model expected to provide the completion We\ntypically set in the range of 10 to 100 as this is how many examples can ﬁt in the model’ context window\nnctx 2048 The main advantages of fewshot are major reduction in the need for taskspeciﬁc data and\nreduced potential to learn an overly narrow distribution from large but narrow ﬁnetuning dataset The main\ndisadvantage is that results from this method have so far been much worse than stateoftheart ﬁnetuned\nmodels Also small amount of task speciﬁc data is still required As indicated by the name fewshot\nlearning as described here for language models is related to fewshot learning as used in other contexts in\nML HYC01 VBL16 – both involve learning based on broad distribution of tasks in this case implicit in\nthe pretraining data and then rapidly adapting to new task\n• OneShot 1S is the same as fewshot except that only one demonstration is allowed in addition to natural\nlanguage description of the task as shown in Figure The reason to distinguish oneshot from fewshot and\nzeroshot below is that it most closely matches the way in which some tasks are communicated to humans\nFor example when asking humans to generate dataset on human worker service for example Mechanical\nTurk it is common to give one demonstration of the task By contrast it is sometimes difﬁcult to communicate\nthe content or format of task if no examples are given'

In [ ]:
len(text)

In [ ]:
from DocumentIndexing.Embedding import embedding_toolkits
from DocumentIndexing.Embedding import text_splitter

In [ ]:
embedding_toolkits.w2v_token_len(text)

In [ ]:
embedding_toolkits.token_length(text)

In [ ]:
text_splitter.split_text_with_langchain(text,512)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
token = model.tokenize(text)

In [ ]:
len(token['input_ids'])

In [ ]:
from transformers import AutoTokenizer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distiluse-base-multilingual-cased-v2')
encoded_input = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')


In [ ]:
input_ids = encoded_input['input_ids'][0] 

In [ ]:
len([token for token in input_ids if token not in tokenizer.all_special_ids])


In [ ]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [ ]:
splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")

In [ ]:
splitter.maximum_tokens_per_chunk = 512

In [ ]:
splitter.count_tokens(text=text)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distiluse-base-multilingual-cased-v2')

In [ ]:
split= splitter.split_text(text=text)

In [ ]:
splitter.count_tokens(text=split[0])

In [ ]:
split

In [ ]:
from sentence_transformers import SentenceTransformer

# Load your SentenceTransformer model
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"  # Replace with your model name
model = SentenceTransformer(model_name)

# Access the underlying Hugging Face model
hf_model = model._first_module().auto_model

# Check the maximum input length
max_length = hf_model.config.max_position_embeddings
print("Maximum input token limit:", max_length)


In [ ]:
import spacy
from spacypdfreader.spacypdfreader import pdf_reader

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = pdf_reader('/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/06063c65-7cc9-4065-8c26-03d8d7aa6379.pdf', nlp)

In [ ]:
doc._.page(1)

In [ ]:
new_text = []
for i in range (1,2):
    for token in doc._.page(i):
        if token.text.endswith('\n'):
        # Remove the line break and do not add a space
            new_text += token.text.rstrip('\n')
        else:
            # Add token text with a following space
            new_text += token.text_with_ws

In [ ]:
new_text

In [ ]:

for token in list(doc._.page(1).sents):
    if token.text.endswith('\n'):
        # Remove the line break and do not add a space
        new_text += token.text.rstrip('\n')
    else:
        # Add token text with a following space
        new_text += token.text_with_ws

In [ ]:
new_text

In [ ]:
sents = list(doc._.page(1).sents)

In [ ]:
sents[0].text_with_ws

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
for page_layout in extract_pages('/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/06063c65-7cc9-4065-8c26-03d8d7aa6379.pdf'):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            print(element.get_text())

In [ ]:
from DocumentProcessing.pdf_processing import pdf_processor

In [ ]:
processor  = pdf_processor.PDFProcessor('/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/06063c65-7cc9-4065-8c26-03d8d7aa6379.pdf')

In [ ]:
pages = processor.extract_text_from_pdf()

In [ ]:
from DocumentIndexing.Embedding import text_splitter

In [ ]:
spliter = text_splitter.TextSplitter()

In [ ]:
spliter.split_text_with_langchain_recursive(pages[1],128)

In [ ]:
spliter.split_text_with_langchain_sentence_transformers(pages[1])

In [ ]:
from DocumentIndexing.Embedding.embedding_toolkits import token_length

In [ ]:
def chunk_sentences(sentences, max_tokens=128):
    chunks = []
    current_chunk = []
    current_token_count = 0

    for sentence in sentences:
        # Tokenize the sentence and count the tokens
        token_count = token_length(sentence.text)

        # If adding this sentence exceeds the max token count, start a new chunk
        if current_token_count + token_count > max_tokens:
            if current_chunk:  # Ensure current chunk is not empty
                chunks.append(' '.join(current_chunk))
            current_chunk = [sentence.text]
            current_token_count = token_count
        else:
            # Add the sentence to the current chunk
            current_chunk.append(sentence.text)
            current_token_count += token_count

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks






In [ ]:
chunks = chunk_sentences(sents)

In [ ]:
[chunks.replace('\n', '') for chunks in chunks]

In [ ]:
def chunk_sentences_with_sentence_overlap(sentences, max_tokens=128, overlap=0):
    chunks = []
    current_chunk = []
    current_token_count = 0
    overlap_sentences = []

    for sentence in sentences:
        token_count = token_length(sentence)

        if current_token_count + token_count > max_tokens:
            if current_chunk:  # Ensure current chunk is not empty
                chunks.append(' '.join(current_chunk))
            # Start new chunk with overlap from previous chunk
            current_chunk = overlap_sentences + [sentence]
            current_token_count = len(' '.join(current_chunk).split())
            # Update overlap sentences
            overlap_sentences = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk[:]
        else:
            # Add the sentence to the current chunk
            current_chunk.append(sentence)
            current_token_count += token_count
            # Update overlap sentences
            if len(current_chunk) > overlap:
                overlap_sentences = current_chunk[-overlap:]

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks





In [ ]:
chunks_o = chunk_sentences(sents, overlap=1)

In [ ]:
[chunks.replace('\n', '') for chunks in chunks_o]

In [ ]:
def chunk_sentences_with_sentence_overlap_generator(sentences, max_tokens=128, overlap=0):
    chunks = []
    current_chunk = []
    current_token_count = 0
    overlap_sentences = []

    for sentence in sentences:
        token_count = token_length(sentence.text)

        if current_token_count + token_count > max_tokens:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            # Start new chunk with overlap from previous chunk
            current_chunk = overlap_sentences + [sentence.text]
            current_token_count = len(' '.join(current_chunk).split())
            # Prepare overlap sentences for next chunk
            overlap_sentences = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk[:]
        else:
            current_chunk.append(sentence.text)
            current_token_count += token_count
            if len(current_chunk) > overlap:
                overlap_sentences = current_chunk[-overlap:]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks



In [ ]:
chunks_g = chunk_sentences_with_sentence_overlap_generator(doc._.page(1).sents, overlap=1)

In [ ]:
[chunks.replace('\n', '') for chunks in chunks_g]

In [1]:
from DocumentIndexing.Embedding.text_splitter import TextSplitter_Spacy
from DocumentProcessing.pdf_processing.pdf_processor import SpacyPdfProcessor

/root/codebox39/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
spacy_processor = SpacyPdfProcessor('/root/gpt_projects/ABoringKnowledgeManagementSystem/WebApp/d42a0cd9-fdfd-4639-a38c-a85b08cd37a7.pdf')

In [ ]:
pages = spacy_processor.extract_text_from_pdf()

In [ ]:
spliter = TextSplitter_Spacy()

In [33]:
split = spliter.chunk_sentences_with_sentence_overlap(pages[1], 128,overlap=1)

In [34]:
split

['BERT: Pre-training of Deep Bidirectional Transformers forLanguage UnderstandingJacob Devlin Ming-Wei Chang Kenton Lee Kristina ToutanovaGoogle AI Language{jacobdevlin,mingweichang,kentonl,kristout}@google.com9102 yaM 42  ]LC.sc [  2v50840.',
 '[  2v50840. 0181:viXraAbstractWe introduce a new language representa-tion model called BERT, which stands forBidirectional Encoder Representations fromTransformers. Unlike recent language repre-sentation models (Peters et al., 2018a; Rad-ford et al., 2018), BERT is designed to pre-train deep bidirectional representations fromunlabeled text by jointly conditioning on bothleft and right context in all layers.',
 'Unlike recent language repre-sentation models (Peters et al., 2018a; Rad-ford et al., 2018), BERT is designed to pre-train deep bidirectional representations fromunlabeled text by jointly conditioning on bothleft and right context in all layers. As a re-sult, the pre-trained BERT model can be ﬁne-tuned with just one additional output lay

In [ ]:
page_generator = spacy_processor.generate_senteces()

In [ ]:
for i in range(spacy_processor.first,spacy_processor.last+1):
    print(spliter.chunk_sentences_with_sentence_overlap(next(page_generator)[1]))

In [22]:
page_generator = spacy_processor.generate_senteces_generator()

In [29]:
page_nb, page = next(page_generator)

In [30]:
split = spliter.chunk_sentences_with_sentence_overlap_generator(page, 128,overlap=1)

In [31]:
page_nb,split

(3,
 ['Figure 1: Overall pre-training and ﬁne-tuning procedures for BERT. Apart from output layers, the same architec-tures are used in both pre-training and ﬁne-tuning. The same pre-trained model parameters are used to initializemodels for different down-stream tasks. During ﬁne-tuning, all parameters are ﬁne-tuned. [CLS] is a specialsymbol added in front of every input example, and [SEP] is a special separator token (e.g. separating ques-tions/answers).',
  '[CLS] is a specialsymbol added in front of every input example, and [SEP] is a special separator token (e.g. separating ques-tions/answers). ing and auto-encoder objectives have been usedfor pre-training such models (Howard and Ruder,2018; Radford et al., 2018; Dai and Le, 2015). 2.3 Transfer Learning from Supervised DataThere has also been work showing effective trans-fer from supervised tasks with large datasets, suchas natural language inference (Conneau et al.,2017) and machine translation (McCann et al.,2017).',
  '2.3 Trans

In [1]:
from DocumentManagement.documents import PDFDocument

In [2]:
pdf = PDFDocument(file_path = '/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/06063c65-7cc9-4065-8c26-03d8d7aa6379.pdf',document_type='research_paper',document_id='123')

In [3]:
pdf.process_document()

KeyError: 1